# 구예보 생성
2020년 08월 29일~2021년 6월 31일

### 기상청 3시간,6시간 간격 구예보 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import warnings
warnings.filterwarnings('ignore')

## Forecast time : 기상청 예보를 발표한 시간 
## forecast_time : 9시간 적용 예보를 발표한 시간 (한국 시간은 UTC + 9) 
## datetime : 예보발표시간에 forecast dateoffset적용한 날씨예측시간(final) 

### <풍력: 화순 동면 예보>


In [ ]:
wind_data_path = '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/동면_구예보'
path_list = sorted(glob(wind_data_path+'/*.csv'))
path_list  

['/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/동면_구예보/동면_3시간기온_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/동면_구예보/동면_6시간강수량_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/동면_구예보/동면_6시간적설_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/동면_구예보/동면_강수형태_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/동면_구예보/동면_강수확률_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/동면_구예보/동면_습도_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/ᄀ

## 3시간 기준 데이터 결합(기온, 강수형태, 강수확률, 습도, 풍속, 풍향, 하늘상태)

In [ ]:
# 3시간 기준 데이터 결합
def preprocessing_3h(path_list):
    #데이터 불러오기
    data_year_temperature = pd.read_csv(path_list[0])
    data_year_precipitationform = pd.read_csv(path_list[3])
    data_year_precipitationprob = pd.read_csv(path_list[4])  
    data_year_humidity = pd.read_csv(path_list[5])
    data_year_windspeed = pd.read_csv(path_list[6])
    data_year_winddirection = pd.read_csv(path_list[7])
    data_year_cloud = pd.read_csv(path_list[8])

    # 데이터 결합 (3시간)
    data_year = data_year_temperature[[' format: day', 'hour', 'forecast']]
    data_year['temperature'] = data_year_temperature[data_year_temperature.columns[-1]] #온도 
    data_year['precipitationform'] = data_year_precipitationform[data_year_precipitationform.columns[-1]] #강수형태
    data_year['precipitationprob'] = data_year_precipitationprob[data_year_precipitationprob.columns[-1]] #강수확률
    data_year['humidity'] = data_year_humidity[data_year_humidity.columns[-1]] #습도
    data_year['windspeed'] = data_year_windspeed[data_year_windspeed.columns[-1]] #풍속  
    data_year['winddirection'] = data_year_winddirection[data_year_winddirection.columns[-1]] #풍향
    data_year['cloud'] = data_year_cloud[data_year_cloud.columns[-1]] #하늘상태

    # 월 구분 행 추출
    month_rows = [-1]
    month_rows.extend(data_year[data_year['hour'].isna()].index)
    month_rows.append(data_year.shape[0]+1)
    
    # 월별 데이터 분리
    month_data = []
    for i in range(len(month_rows)-1):
        month_data.append(data_year.loc[month_rows[i]+1:month_rows[i+1]-1])
     
    # 예보 시간 생성 및 데이터 결합
    data_fcst_concat = pd.DataFrame(columns=['Forecast time', 'forecast','temperature','precipitationform', 'precipitationprob', 'humidity','windspeed','winddirection','cloud'])
    for i, df in enumerate(month_data):
        month = i+8
        year = 2020
        if month == 13:
            month = 1
            year += 1
        elif month == 14:
            month = 2
            year += 1
        elif month == 15:
            month = 3
            year += 1
        elif month == 16:
            month = 4
            year += 1
        elif month == 17:
            month = 5
            year += 1
        elif month == 18:
            month = 6
            year += 1
        elif month == 19:
            month = 7
            year += 1
        elif month == 20:
            month = 8
            year += 1
        elif month == 21:
            month = 9
            year += 1
       

        date = f'{year}-{month}-' + df[' format: day'].str.split(' ').str[-1] + ' ' + (df['hour'].astype(int)//100).astype(str) + ':00'
        
        # 시간단위 UTC => GMT
        
        data_fcst = pd.DataFrame(columns=['Forecast time', 'forecast','temperature','precipitationform', 'precipitationprob', 'humidity','windspeed','winddirection','cloud'])
        data_fcst['Forecast time'] = date
        
        date = pd.to_datetime(date) + pd.DateOffset(hours=9)
        data_fcst['forecast_time'] = date
        data_fcst['forecast'] = df['forecast'].copy()
        data_fcst['temperature'] = df['temperature'].copy()
        data_fcst['precipitationform'] = df['precipitationform'].copy()
        data_fcst['precipitationprob'] = df['precipitationprob'].copy()
        data_fcst['humidity'] = df['humidity'].copy()
        data_fcst['windspeed'] = df['windspeed'].copy()
        data_fcst['winddirection'] = df['winddirection'].copy()
        data_fcst['cloud'] = df['cloud'].copy()
        #data_fcst[['forecast', 'temperature','precipitationform','precipitationprob,' 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']] = df[['forecast', 'Temperature','PrecipitationForm','PrecipitationProb', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]

        data_fcst_concat = pd.concat([data_fcst_concat, data_fcst])
        
    return data_fcst_concat

In [ ]:
#14시 예보 기준
def make_14(df): 
    df['forecast_time'] = pd.to_datetime(df['forecast_time'])
    fcst_14 = df[df['forecast_time'].dt.hour==14]
    fcst_14 = fcst_14[(fcst_14['forecast']>=10)&(fcst_14['forecast']<=33)]
    def to_date(x):
        return pd.DateOffset(hours=x)
    fcst_14['datetime'] = fcst_14['forecast_time'] + fcst_14['forecast'].map(to_date)
    
    fcst_14_ = pd.DataFrame()
    fcst_14_['datetime'] = pd.date_range(start='2020-08-02 00:00:00', end='2021-07-01 21:00:00', freq='H')
    
    fcst_14_ = pd.merge(fcst_14_, fcst_14, on='datetime', how='outer')
    fcst_14_df = fcst_14_[['datetime','temperature','precipitationform', 'precipitationprob', 'humidity','windspeed','winddirection','cloud']]
    fcst_14_df['temperature'] = fcst_14_df['temperature'].astype('float').interpolate()
    fcst_14_df['precipitationform'] = fcst_14_df['precipitationform'].astype('float').interpolate()
    fcst_14_df['precipitationprob'] = fcst_14_df['precipitationprob'].astype('float').interpolate()
    fcst_14_df['humidity'] = fcst_14_df['humidity'].astype('float').interpolate()
    fcst_14_df['windspeed'] = fcst_14_df['windspeed'].astype('float').interpolate()
    fcst_14_df['winddirection'] = fcst_14_df['winddirection'].astype('float').interpolate()
    fcst_14_df['cloud'] = fcst_14_df['cloud'].astype('float').interpolate()
    return fcst_14_df 

In [ ]:
wind_df = preprocessing_3h(path_list)
wind_df

,Forecast time,forecast,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,forecast_time
0,2020-8-1 2:00,4.0,30.0,0.0,30.0,70.0,2.7,193.0,4.0,2020-08-01 11:00:00
1,2020-8-1 2:00,7.0,29.0,1.0,60.0,75.0,2.4,185.0,4.0,2020-08-01 11:00:00
2,2020-8-1 2:00,10.0,25.0,0.0,20.0,95.0,1.6,176.0,3.0,2020-08-01 11:00:00
3,2020-8-1 2:00,13.0,24.0,0.0,20.0,95.0,1.4,180.0,3.0,2020-08-01 11:00:00
4,2020-8-1 2:00,16.0,24.0,0.0,30.0,95.0,1.4,192.0,4.0,2020-08-01 11:00:00
...,...,...,...,...,...,...,...,...,...,...
49437,2021-6-30 23:00,52.0,25.0,0.0,30.0,80.0,3.7,114.0,4.0,2021-07-01 08:00:00
49438,2021-6-30 23:00,55.0,25.0,1.0,64.0,80.0,3.9,114.0,4.0,2021-07-01 08:00:00
49439,2021-6-30 23:00,58.0,24.0,1.0,60.0,85.0,3.7,121.0,4.0,2021-07-01 08:00:00
49440,2021-6-30 23:00,61.0,24.0,1.0,63.0,90.0,3.7,171.0,4.0,2021-07-01 08:00:00


In [ ]:
wind_fcst_14 = make_14(wind_df)
wind_fcst_14

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud
0,2020-08-02 00:00:00,24.000000,0.0,20.000000,95.000000,1.400000,180.000000,3.000000
1,2020-08-02 01:00:00,24.000000,0.0,23.333333,95.000000,1.400000,184.000000,3.333333
2,2020-08-02 02:00:00,24.000000,0.0,26.666667,95.000000,1.400000,188.000000,3.666667
3,2020-08-02 03:00:00,24.000000,0.0,30.000000,95.000000,1.400000,192.000000,4.000000
4,2020-08-02 04:00:00,24.000000,0.0,26.666667,95.000000,1.400000,190.666667,3.666667
...,...,...,...,...,...,...,...,...
8009,2021-07-01 17:00:00,29.666667,0.0,3.333333,56.666667,2.133333,147.666667,1.666667
8010,2021-07-01 18:00:00,29.000000,0.0,0.000000,60.000000,2.300000,155.000000,1.000000
8011,2021-07-01 19:00:00,27.666667,0.0,6.666667,68.333333,2.133333,154.333333,1.666667
8012,2021-07-01 20:00:00,26.333333,0.0,13.333333,76.666667,1.966667,153.666667,2.333333


## 6시간 기준 데이터 결합(강수량, 적설)

In [ ]:
def preprocessing_6h(path_list):
    #데이터 불러오기
    data_year_precipitation = pd.read_csv(path_list[1])
    data_year_snow = pd.read_csv(path_list[2])

    # 데이터 결합 (6시간)
    data_year = data_year_precipitation[[' format: day', 'hour', 'forecast']]
    data_year['precipitation'] = data_year_precipitation[data_year_precipitation.columns[-1]] #강수량
    data_year['snow'] = data_year_snow[data_year_snow.columns[-1]] #적설
    # 월 구분 행 추출
    month_rows = [-1]
    month_rows.extend(data_year[data_year['hour'].isna()].index)
    month_rows.append(data_year.shape[0]+1)
    
    # 월별 데이터 분리
    month_data = []
    for i in range(len(month_rows)-1):
        month_data.append(data_year.loc[month_rows[i]+1:month_rows[i+1]-1])
     
    # 예보 시간 생성 및 데이터 결합
    data_fcst_concat = pd.DataFrame(columns=['Forecast time', 'forecast', 'precipitation','snow'])
    for i, df in enumerate(month_data):
        month = i+8
        year = 2020
        if month == 13:
            month = 1
            year += 1
        elif month == 14:
            month = 2
            year += 1
        elif month == 15:
            month = 3
            year += 1
        elif month == 16:
            month = 4
            year += 1
        elif month == 17:
            month = 5
            year += 1
        elif month == 18:
            month = 6
            year += 1
        elif month == 19:
            month = 7
            year += 1
        elif month == 20:
            month = 8
            year += 1
        elif month == 21:
            month = 9
            year += 1

        date = f'{year}-{month}-' + df[' format: day'].str.split(' ').str[-1] + ' ' + (df['hour'].astype(int)//100).astype(str) + ':00'
        
        # 시간단위 UTC => GMT
        data_fcst = pd.DataFrame(columns=['Forecast time', 'forecast', 'precipitation','snow'])
        data_fcst['Forecast time'] = date
        date = pd.to_datetime(date) + pd.DateOffset(hours=9)
        data_fcst['forecast_time'] = date
        data_fcst[['forecast', 'precipitation','snow']] = df[['forecast',  'precipitation','snow']]

        data_fcst_concat = pd.concat([data_fcst_concat, data_fcst])
        
    return data_fcst_concat           

In [ ]:
def make_14_6hr(df): 
    df['forecast_time'] = pd.to_datetime(df['forecast_time'])
    fcst_14_p = df[df['forecast_time'].dt.hour==14]
    fcst_14_p = fcst_14_p[(fcst_14_p['forecast']>=10)&(fcst_14_p['forecast']<=33)]
    def to_date(x):
        return pd.DateOffset(hours=x)
    fcst_14_p['datetime'] = fcst_14_p['forecast_time'] + fcst_14_p['forecast'].map(to_date)
    fcst_14_ = pd.DataFrame()
    fcst_14_['datetime'] = pd.date_range(start='2020-08-02 00:00:00', end='2021-07-01 18:00:00', freq='H')
    fcst_14_ = pd.merge(fcst_14_, fcst_14_p, on='datetime', how='outer')
    fcst_14_df_p = fcst_14_[['datetime',  'precipitation','snow']]
    
    fcst_14_df_p['precipitation'] = fcst_14_df_p['precipitation'].astype('float').interpolate()
    fcst_14_df_p['snow'] = fcst_14_df_p['snow'].astype('float').interpolate()
    return fcst_14_df_p

In [ ]:
wind_df2 = preprocessing_6h(path_list)
wind_fcst_14_df_p = make_14_6hr(wind_df2)

In [ ]:
wind_fcst_14_df_p

,datetime,precipitation,snow
0,2020-08-02 00:00:00,0.0,0.0
1,2020-08-02 01:00:00,0.0,0.0
2,2020-08-02 02:00:00,0.0,0.0
3,2020-08-02 03:00:00,0.0,0.0
4,2020-08-02 04:00:00,0.0,0.0
...,...,...,...
8006,2021-07-01 14:00:00,0.0,0.0
8007,2021-07-01 15:00:00,0.0,0.0
8008,2021-07-01 16:00:00,0.0,0.0
8009,2021-07-01 17:00:00,0.0,0.0


In [ ]:
wind_fcst_14

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud
0,2020-08-02 00:00:00,24.000000,0.0,20.000000,95.000000,1.400000,180.000000,3.000000
1,2020-08-02 01:00:00,24.000000,0.0,23.333333,95.000000,1.400000,184.000000,3.333333
2,2020-08-02 02:00:00,24.000000,0.0,26.666667,95.000000,1.400000,188.000000,3.666667
3,2020-08-02 03:00:00,24.000000,0.0,30.000000,95.000000,1.400000,192.000000,4.000000
4,2020-08-02 04:00:00,24.000000,0.0,26.666667,95.000000,1.400000,190.666667,3.666667
...,...,...,...,...,...,...,...,...
8009,2021-07-01 17:00:00,29.666667,0.0,3.333333,56.666667,2.133333,147.666667,1.666667
8010,2021-07-01 18:00:00,29.000000,0.0,0.000000,60.000000,2.300000,155.000000,1.000000
8011,2021-07-01 19:00:00,27.666667,0.0,6.666667,68.333333,2.133333,154.333333,1.666667
8012,2021-07-01 20:00:00,26.333333,0.0,13.333333,76.666667,1.966667,153.666667,2.333333


In [ ]:
wind_fcst_14_f = pd.merge(wind_fcst_14, wind_fcst_14_df_p, on='datetime')

In [ ]:
wind_fcst_14_f

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow
0,2020-08-02 00:00:00,24.000000,0.0,20.000000,95.000000,1.400000,180.000000,3.000000,0.0,0.0
1,2020-08-02 01:00:00,24.000000,0.0,23.333333,95.000000,1.400000,184.000000,3.333333,0.0,0.0
2,2020-08-02 02:00:00,24.000000,0.0,26.666667,95.000000,1.400000,188.000000,3.666667,0.0,0.0
3,2020-08-02 03:00:00,24.000000,0.0,30.000000,95.000000,1.400000,192.000000,4.000000,0.0,0.0
4,2020-08-02 04:00:00,24.000000,0.0,26.666667,95.000000,1.400000,190.666667,3.666667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
8006,2021-07-01 14:00:00,30.333333,0.0,6.666667,51.666667,1.833333,124.000000,2.333333,0.0,0.0
8007,2021-07-01 15:00:00,31.000000,0.0,10.000000,50.000000,1.800000,133.000000,3.000000,0.0,0.0
8008,2021-07-01 16:00:00,30.333333,0.0,6.666667,53.333333,1.966667,140.333333,2.333333,0.0,0.0
8009,2021-07-01 17:00:00,29.666667,0.0,3.333333,56.666667,2.133333,147.666667,1.666667,0.0,0.0


In [ ]:
#*datetime 기준
#구예보: 2020-08-29 00:00:00~2020.06.30 23:00:00
wind_fcst_14_f2 = wind_fcst_14_f[(wind_fcst_14_f['datetime']>='2020-08-29 00:00:00')&(wind_fcst_14_f['datetime']<='2021-06-30 23:00:00')].copy()
wind_fcst_14_f2.reset_index(drop=True, inplace=True)
wind_fcst_14_f2

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow
0,2020-08-29 00:00:00,26.000000,0.0,30.000000,95.000000,1.400000,126.000000,4.000000,0.0,0.0
1,2020-08-29 01:00:00,26.000000,0.0,30.000000,95.000000,1.333333,125.666667,4.000000,0.0,0.0
2,2020-08-29 02:00:00,26.000000,0.0,30.000000,95.000000,1.266667,125.333333,4.000000,0.0,0.0
3,2020-08-29 03:00:00,26.000000,0.0,30.000000,95.000000,1.200000,125.000000,4.000000,0.0,0.0
4,2020-08-29 04:00:00,26.000000,0.0,30.000000,95.000000,1.266667,123.333333,4.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7339,2021-06-30 19:00:00,26.666667,0.0,10.000000,66.666667,1.733333,147.000000,2.333333,0.0,0.0
7340,2021-06-30 20:00:00,25.333333,0.0,10.000000,73.333333,1.366667,149.000000,1.666667,0.0,0.0
7341,2021-06-30 21:00:00,24.000000,0.0,10.000000,80.000000,1.000000,151.000000,1.000000,0.0,0.0
7342,2021-06-30 22:00:00,22.666667,0.0,6.666667,83.333333,1.033333,130.666667,1.000000,0.0,0.0


In [ ]:
wind_fcst_14_f2.to_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/wind_old_forecast14.csv', index=False) 

### <태양광: 인천 청라3동 예보>

In [ ]:
solar_data_path = '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/청라3동_구예보'
path_list2 = sorted(glob(solar_data_path+'/*.csv'))
path_list2  

['/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/청라3동_구예보/청라3동_3시간기온_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/청라3동_구예보/청라3동_6시간강수량_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/청라3동_구예보/청라3동_6시간적설_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/청라3동_구예보/청라3동_강수형태_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/청라3동_구예보/청라3동_강수확률_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/raw_data/기상청/청라3동_구예보/청라3동_습도_202008_202106.csv',
 '/content/drive/MyDrive/Colab Notebooks/AIfactory_에너

In [ ]:
solar_df = preprocessing_3h(path_list2)
solar_df

,Forecast time,forecast,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,forecast_time
0,2020-8-1 2:00,4.0,25.0,1.0,80.0,90.0,3.1,171.0,4.0,2020-08-01 11:00:00
1,2020-8-1 2:00,7.0,25.0,1.0,80.0,95.0,2.9,178.0,4.0,2020-08-01 11:00:00
2,2020-8-1 2:00,10.0,24.0,1.0,70.0,90.0,2.5,192.0,4.0,2020-08-01 11:00:00
3,2020-8-1 2:00,13.0,24.0,1.0,70.0,95.0,2.3,198.0,4.0,2020-08-01 11:00:00
4,2020-8-1 2:00,16.0,24.0,1.0,80.0,95.0,2.3,198.0,4.0,2020-08-01 11:00:00
...,...,...,...,...,...,...,...,...,...,...
49437,2021-6-30 23:00,52.0,27.0,0.0,30.0,60.0,4.2,111.0,4.0,2021-07-01 08:00:00
49438,2021-6-30 23:00,55.0,27.0,0.0,30.0,60.0,5.6,98.0,4.0,2021-07-01 08:00:00
49439,2021-6-30 23:00,58.0,25.0,0.0,30.0,75.0,5.9,126.0,4.0,2021-07-01 08:00:00
49440,2021-6-30 23:00,61.0,23.0,1.0,100.0,80.0,5.4,117.0,4.0,2021-07-01 08:00:00


In [ ]:
solar_fcst_14 = make_14(solar_df)
solar_fcst_14

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud
0,2020-08-02 00:00:00,24.000000,1.0,70.000000,95.000000,2.300000,198.000000,4.0
1,2020-08-02 01:00:00,24.000000,1.0,73.333333,95.000000,2.300000,198.000000,4.0
2,2020-08-02 02:00:00,24.000000,1.0,76.666667,95.000000,2.300000,198.000000,4.0
3,2020-08-02 03:00:00,24.000000,1.0,80.000000,95.000000,2.300000,198.000000,4.0
4,2020-08-02 04:00:00,24.000000,1.0,80.000000,95.000000,2.266667,196.333333,4.0
...,...,...,...,...,...,...,...,...
8009,2021-07-01 17:00:00,27.666667,0.0,0.000000,63.333333,3.233333,278.666667,1.0
8010,2021-07-01 18:00:00,27.000000,0.0,0.000000,65.000000,3.200000,292.000000,1.0
8011,2021-07-01 19:00:00,26.000000,0.0,0.000000,68.333333,2.666667,298.000000,1.0
8012,2021-07-01 20:00:00,25.000000,0.0,0.000000,71.666667,2.133333,304.000000,1.0


In [ ]:
solar_df2 = preprocessing_6h(path_list2)
solar_fcst_14_df_p = make_14_6hr(solar_df2)

In [ ]:
solar_fcst_14_df_p

,datetime,precipitation,snow
0,2020-08-02 00:00:00,20.000000,0.0
1,2020-08-02 01:00:00,18.333333,0.0
2,2020-08-02 02:00:00,16.666667,0.0
3,2020-08-02 03:00:00,15.000000,0.0
4,2020-08-02 04:00:00,13.333333,0.0
...,...,...,...
8006,2021-07-01 14:00:00,0.000000,0.0
8007,2021-07-01 15:00:00,0.000000,0.0
8008,2021-07-01 16:00:00,0.000000,0.0
8009,2021-07-01 17:00:00,0.000000,0.0


In [ ]:
solar_fcst_14_f = pd.merge(solar_fcst_14, solar_fcst_14_df_p, on='datetime')

In [ ]:
solar_fcst_14_f

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow
0,2020-08-02 00:00:00,24.000000,1.0,70.000000,95.000000,2.300000,198.000000,4.0,20.000000,0.0
1,2020-08-02 01:00:00,24.000000,1.0,73.333333,95.000000,2.300000,198.000000,4.0,18.333333,0.0
2,2020-08-02 02:00:00,24.000000,1.0,76.666667,95.000000,2.300000,198.000000,4.0,16.666667,0.0
3,2020-08-02 03:00:00,24.000000,1.0,80.000000,95.000000,2.300000,198.000000,4.0,15.000000,0.0
4,2020-08-02 04:00:00,24.000000,1.0,80.000000,95.000000,2.266667,196.333333,4.0,13.333333,0.0
...,...,...,...,...,...,...,...,...,...,...
8006,2021-07-01 14:00:00,28.666667,0.0,0.000000,61.666667,2.533333,237.666667,1.0,0.000000,0.0
8007,2021-07-01 15:00:00,29.000000,0.0,0.000000,60.000000,3.300000,252.000000,1.0,0.000000,0.0
8008,2021-07-01 16:00:00,28.333333,0.0,0.000000,61.666667,3.266667,265.333333,1.0,0.000000,0.0
8009,2021-07-01 17:00:00,27.666667,0.0,0.000000,63.333333,3.233333,278.666667,1.0,0.000000,0.0


In [ ]:
#*datetime 기준
#구예보: 2020-08-29 00:00:00~2020.06.30 23:00:00
solar_fcst_14_f2 = solar_fcst_14_f[(solar_fcst_14_f['datetime']>='2020-08-29 00:00:00')&(solar_fcst_14_f['datetime']<='2021-06-30 23:00:00')].copy()
solar_fcst_14_f2.reset_index(drop=True, inplace=True)
solar_fcst_14_f2

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow
0,2020-08-29 00:00:00,26.000000,0.0,20.000000,90.000000,0.900000,174.000000,3.000000,0.0,0.0
1,2020-08-29 01:00:00,26.000000,0.0,23.333333,91.666667,0.900000,176.000000,3.333333,0.0,0.0
2,2020-08-29 02:00:00,26.000000,0.0,26.666667,93.333333,0.900000,178.000000,3.666667,0.0,0.0
3,2020-08-29 03:00:00,26.000000,0.0,30.000000,95.000000,0.900000,180.000000,4.000000,0.0,0.0
4,2020-08-29 04:00:00,26.000000,0.0,30.000000,95.000000,0.900000,138.000000,4.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7339,2021-06-30 19:00:00,25.000000,0.0,0.000000,73.333333,2.766667,284.666667,1.000000,0.0,0.0
7340,2021-06-30 20:00:00,24.000000,0.0,0.000000,76.666667,1.933333,296.333333,1.000000,0.0,0.0
7341,2021-06-30 21:00:00,23.000000,0.0,0.000000,80.000000,1.100000,308.000000,1.000000,0.0,0.0
7342,2021-06-30 22:00:00,22.666667,0.0,10.000000,81.666667,0.800000,256.333333,2.000000,0.0,0.0


In [ ]:
solar_fcst_14_f2.to_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/solar_old_forecast14.csv', index=False) 